<a href="https://colab.research.google.com/github/Tejasri1557/ADM_2025/blob/main/2303a51557_ASSN_06_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/content/tweets.csv.zip")

In [3]:
df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import re

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [14]:
from nltk.stem import WordNetLemmatizer

In [15]:
import nltk

In [16]:
import string

In [33]:
nltk.download('punct')
nltk.download('stopwords')
nltk.download('punct_tab')
nltk.download('wordnet')

[nltk_data] Error loading punct: Package 'punct' not found in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading punct_tab: Package 'punct_tab' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
stop_words = set(stopwords.words('english'))

In [35]:
lemmatizer = WordNetLemmatizer()

In [41]:
# Make sure to run the NLTK download cell (zvSNFrcTSm_0) before running this cell.
nltk.download('punkt_tab')
def preprocess_text(text):
    text=text.lower()
    text=text.translate(str.maketrans('', '', string.punctuation))
    text=re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens=word_tokenize(text)
    stop_words=set(stopwords.words('english'))
    tokens=[word for word in tokens if word not in stop_words]
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)
df["cleaned_text"]=df["text"].astype(str).apply(preprocess_text)
print(df)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


          id  keyword                 location  \
0          0   ablaze                      NaN   
1          1   ablaze                      NaN   
2          2   ablaze            New York City   
3          3   ablaze           Morgantown, WV   
4          4   ablaze                      NaN   
...      ...      ...                      ...   
11365  11365  wrecked  Blue State in a red sea   
11366  11366  wrecked               arohaonces   
11367  11367  wrecked                       🇵🇭   
11368  11368  wrecked           auroraborealis   
11369  11369  wrecked                      NaN   

                                                    text  target  \
0      Communal violence in Bhainsa, Telangana. "Ston...       1   
1      Telangana: Section 144 has been imposed in Bha...       1   
2      Arsonist sets cars ablaze at dealership https:...       1   
3      Arsonist sets cars ablaze at dealership https:...       1   
4      "Lord Jesus, your love brings freedom and pard...   

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2),max_features=20000)
X=vectorizer.fit_transform(df["cleaned_text"])
y=df["target"]

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout,Conv1D,GlobalMaxPooling1D,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [46]:
mlp = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
mlp.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
mlp.fit(X_train.toarray(),y_train,validation_data=(X_test.toarray(),y_test),epochs=5,batch_size=64)

Epoch 1/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.9957 - loss: 0.0154 - val_accuracy: 0.8914 - val_loss: 0.5000
Epoch 2/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9959 - loss: 0.0108 - val_accuracy: 0.8958 - val_loss: 0.5349
Epoch 3/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.9951 - loss: 0.0125 - val_accuracy: 0.8958 - val_loss: 0.5619
Epoch 4/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9968 - loss: 0.0087 - val_accuracy: 0.8931 - val_loss: 0.5658
Epoch 5/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.9982 - loss: 0.0062 - val_accuracy: 0.8953 - val_loss: 0.5892


In [59]:
loss, accuracy = mlp.evaluate(X_test.toarray(), y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9008 - loss: 0.5592
Test Loss: 0.5892
Test Accuracy: 0.8953


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_text'])
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")

sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the sequences to a fixed length
max_length = max([len(seq) for seq in sequences])
print(f"Maximum sequence length: {max_length}")

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
# Split the padded sequences into training and testing sets
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)